In [ ]:
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True

# 입력 이미지 불러오기
input_image = Image.open('/content/KakaoTalk_20231122_191240087_09.jpg')

# 이미지 크기 변경 (예: 512x512)
new_size = (768, 1024)
input_image = input_image.resize(new_size)

input_image = input_image.convert("RGB")

# 입력 이미지를 딥러닝 모델에 넣을 수 있는 형태로 전처리 수행
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

# 딥러닝 불러오기
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

# 딥러닝에 입력 이미지 넣고 예측 결과 가져오기
output = model(input_batch)['out'][0]
output_predictions = output.argmax(0)

# 결과를 칼라로 표현하도록 색상표 만들기
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# 결과 이미지 생성하기
r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(new_size)
r.putpalette(colors)

# 결과 이미지 표출하기
plt.imshow(r)
plt.axis('off')
plt.show()


In [ ]:
#시멘틱세그멘테이션 결과
plt.imshow(r)
plt.axis('off')

In [ ]:
#사람부분(파란색)만 1 흰색으로 출력 나머진 0 검정
import numpy as np
rarray = np.array(r)
result = np.where(rarray == 15, 1, 0)
plt.imshow(result,"gray")
plt.axis('off')

In [ ]:
#원본이미지에서 위의 1인부분만 출력
import cv2
image = np.array(input_image)
masked_image = image * result[:, :, None]
# 결과 이미지 표시
plt.imshow(masked_image)
plt.axis('off')
masked_image1 = cv2.cvtColor(masked_image.astype(np.uint8), cv2.COLOR_RGB2BGR)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Colab에서 이미지를 표시하기 위한 함수

def extract_skin_color(image, lower_skin_threshold, upper_skin_threshold):
    # BGR을 HSV로 변환
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 살색을 정의하는 HSV 범위
    lower_skin = np.array(lower_skin_threshold, dtype=np.uint8)
    upper_skin = np.array(upper_skin_threshold, dtype=np.uint8)

    # HSV 이미지에서 살색에 해당하는 부분 추출
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # 원본 이미지에서 살색 부분만 추출
    result = cv2.bitwise_and(image, image, mask=skin_mask)

    # 결과를 Colab에서 표시
    cv2_imshow(result)





In [ ]:
#황색 범위 설정, 최종 결과
lower_skin_threshold = [8, 80, 150]  # 예시값, 조절 가능
upper_skin_threshold = [18, 130, 220]  # 예시값, 조절 가능
extract_skin_color(masked_image1, lower_skin_threshold, upper_skin_threshold)